In [1]:
import pandas as pd
import sklearn as sk
import tensorflow as tf
import keras
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

2023-08-21 08:46:12.229136: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
df = pd.read_csv('cdc_sample.csv')
df.dropna(inplace=True)
df.head()

,SEQN,SPATTYPE,SPAMANU,SPAFACT,SPAPOS,SPAPLAT,SPAACC,SPAQEFF,SPXPTS,SPXRAW
0,66208,Pre,3,1.080,Standing,Y,Y,A,1026,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
1,62541,Pre,7,1.080,Standing,N,Y,C,452,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
2,71628,Pre,2,1.091,Standing,Y,Y,A,1081,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
3,64470,Pre,1,1.079,Standing,N,N,D,388,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
4,65928,Pre,1,1.085,Standing,N,Y,C,179,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."


In [3]:
# Turn raw data curve into list of integers
df["SPXRAW"] = df.apply(lambda row : np.array([int(num) for num in row['SPXRAW'].split(",")]), axis=1)

In [5]:
# Normalization of vector to same SPXRAW length
x = df["SPXRAW"].to_numpy()
n_steps = max([trial.shape[0] for trial in x])

# Pad 0s in the back
#x_padded = np.array([np.pad(trial, (0, n_steps - trial.shape[0]), mode='constant') for trial in x])

# Pad 0s in front and back
x_padded = np.array([np.pad(trial, 
                            ((n_steps-trial.shape[0])//2, (n_steps - trial.shape[0]) - (n_steps-trial.shape[0])//2), 
                             mode='constant') for trial in x])

# Pad 0x in the front
#x_padded = np.array([np.pad(trial, (n_steps - trial.shape[0], 0), mode='constant') for trial in x])

scaler = preprocessing.MinMaxScaler()  # initialize the scaler
x_scaled = scaler.fit_transform(x_padded)

In [6]:
# Convert labels to numerical values
label_encoder = preprocessing.LabelEncoder()
y = label_encoder.fit_transform(df['SPAQEFF'])

#convert numbered classes to be treated as categorical variable
n_classes = len(np.unique(y))
#y_categorical = to_categorical(y, n_classes)

In [7]:
# Set Random Seed
random_seed = 204
np.random.seed(random_seed)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.3, random_state=random_seed)

In [9]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [10]:
# Architecture
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [11]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=6,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    #optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

max_epochs = 150

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=max_epochs,
    batch_size=8,
    callbacks=callbacks,
)



Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 2044, 1)]    0           []                               
                                                                                                  
 layer_normalization_12 (LayerN  (None, 2044, 1)     2           ['input_2[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 multi_head_attention_6 (MultiH  (None, 2044, 1)     7169        ['layer_normalization_12[0][0]', 
 eadAttention)                                                    'layer_normalization_12[0][0]'] 
                                                                                            

 ambda)                                                           'tf.__operators__.add_17[0][0]']
                                                                                                  
 layer_normalization_19 (LayerN  (None, 2044, 1)     2           ['tf.__operators__.add_18[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv1d_18 (Conv1D)             (None, 2044, 4)      8           ['layer_normalization_19[0][0]'] 
                                                                                                  
 dropout_20 (Dropout)           (None, 2044, 4)      0           ['conv1d_18[0][0]']              
                                                                                                  
 conv1d_19 (Conv1D)             (None, 2044, 1)      5           ['dropout_20[0][0]']             
          

2023-08-19 19:10:37.033739: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-08-19 19:10:37.634386: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-19 19:10:37.634996: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-19 19:10:37.635036: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-08-19 19:10:37.635723: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-19 19:10:37.635812: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


700/700 [==============================] - 809s 1s/step - loss: 1.1532 - sparse_categorical_accuracy: 0.5280 - val_loss: 1.1582 - val_sparse_categorical_accuracy: 0.5157
Epoch 2/150
700/700 [==============================] - 812s 1s/step - loss: 1.0854 - sparse_categorical_accuracy: 0.5546 - val_loss: 1.0326 - val_sparse_categorical_accuracy: 0.5664
Epoch 3/150
700/700 [==============================] - 814s 1s/step - loss: 1.0374 - sparse_categorical_accuracy: 0.5821 - val_loss: 0.9902 - val_sparse_categorical_accuracy: 0.5886
Epoch 4/150
700/700 [==============================] - 813s 1s/step - loss: 0.9965 - sparse_categorical_accuracy: 0.6037 - val_loss: 0.9698 - val_sparse_categorical_accuracy: 0.6043
Epoch 5/150
700/700 [==============================] - 813s 1s/step - loss: 0.9668 - sparse_categorical_accuracy: 0.6145 - val_loss: 0.9286 - val_sparse_categorical_accuracy: 0.6450
Epoch 6/150
700/700 [==============================] - 809s 1s/step - loss: 0.9307 - sparse_categorica

Epoch 46/150
700/700 [==============================] - 810s 1s/step - loss: 0.6732 - sparse_categorical_accuracy: 0.7504 - val_loss: 0.7282 - val_sparse_categorical_accuracy: 0.7421
Epoch 47/150
700/700 [==============================] - 811s 1s/step - loss: 0.6781 - sparse_categorical_accuracy: 0.7538 - val_loss: 0.7220 - val_sparse_categorical_accuracy: 0.7329
Epoch 48/150
700/700 [==============================] - 812s 1s/step - loss: 0.6603 - sparse_categorical_accuracy: 0.7580 - val_loss: 0.7266 - val_sparse_categorical_accuracy: 0.7286
Epoch 49/150
700/700 [==============================] - 807s 1s/step - loss: 0.6617 - sparse_categorical_accuracy: 0.7598 - val_loss: 0.7235 - val_sparse_categorical_accuracy: 0.7379
Epoch 50/150
700/700 [==============================] - 810s 1s/step - loss: 0.6640 - sparse_categorical_accuracy: 0.7611 - val_loss: 0.7275 - val_sparse_categorical_accuracy: 0.7429
Epoch 51/150
700/700 [==============================] - 816s 1s/step - loss: 0.6611 -

In [1]:
file_path = 'model_6_88.keras'
model.save(file_path)

NameError: name 'model' is not defined

In [11]:
# load again from certain epoch
file_path = './split/model_6_70.keras'
model = load_model(file_path)

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=80,
    batch_size=8,
    callbacks=callbacks,
)

2023-08-21 08:47:31.314466: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-21 08:47:31.320201: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-21 08:47:31.320480: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-21 08:47:31.321013: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

Epoch 1/80


2023-08-21 08:47:36.470320: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-08-21 08:47:37.096897: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-21 08:47:37.100745: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-21 08:47:37.100787: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-08-21 08:47:37.104277: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-21 08:47:37.104402: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


700/700 [==============================] - 821s 1s/step - loss: 0.7856 - sparse_categorical_accuracy: 0.7121 - val_loss: 0.7707 - val_sparse_categorical_accuracy: 0.7129
Epoch 2/80
700/700 [==============================] - 820s 1s/step - loss: 0.7860 - sparse_categorical_accuracy: 0.7121 - val_loss: 0.7711 - val_sparse_categorical_accuracy: 0.7121
Epoch 3/80
700/700 [==============================] - 820s 1s/step - loss: 0.7850 - sparse_categorical_accuracy: 0.7120 - val_loss: 0.7692 - val_sparse_categorical_accuracy: 0.7150
Epoch 4/80
700/700 [==============================] - 821s 1s/step - loss: 0.7751 - sparse_categorical_accuracy: 0.7155 - val_loss: 0.7617 - val_sparse_categorical_accuracy: 0.7121
Epoch 5/80
700/700 [==============================] - 820s 1s/step - loss: 0.7846 - sparse_categorical_accuracy: 0.7143 - val_loss: 0.7689 - val_sparse_categorical_accuracy: 0.7157
Epoch 6/80
700/700 [==============================] - 820s 1s/step - loss: 0.7790 - sparse_categorical_acc

In [12]:
file_path = 'model_6_150.keras'
model.save(file_path)

In [ ]:
model = load_model('model_93.keras')

# Evaluate the performance of the model on the testing set
loss, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))